In [ ]:
# Uncomment to upgrade packages
# !pip3 install pandas --user --upgrade --quiet
# !pip3 install numpy --user --upgrade --quiet
# !pip3 install scipy --user --upgrade --quiet
# !pip3 install statsmodels --user --upgrade --quiet
# !pip3 install seaborn --user --upgrade --quiet
# !pip3 install matplotlib --user --upgrade --quiet
# !pip3 install scikit-learn --user --upgrade 
# !pip install scikit-optimize --user --quiet
!pip install apafib --upgrade --user --quiet

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from time import time
from datetime import timedelta
init_time = time()

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split,  KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn import set_config

from yellowbrick.regressor import AlphaSelection

from statsmodels.genmod.generalized_linear_model import GLM
import statsmodels.api as sm
from scipy import stats

import warnings

set_config(display='text')
warnings.filterwarnings('ignore')

sns.set()
pd.set_option('display.precision', 3)

# Ejercicio práctico de regresión lineal

En este ejercicio trabajaremos con el conjunto de datos King County House Data.

Podeis encontrar la documentación de este conjunto de datos aqui (https://www.kaggle.com/datasets/harlfoxem/housesalesprediction)



El podéis aplicar como mínimo los siguientes pasos:
- [ ] Visualización mínima de los datos
- [ ] División en conjunto de entrenamiento y test
- [ ] Estandarización de las variables
- [ ] Regresión lineal, validación cruzada, análisis del modelo
- [ ] Regresión lineal regularizada (Ridge/LASSO), selección del parametro de regularización, análisis del modelo

In [13]:
from apafib import load_king_county_houses

data = load_king_county_houses()
data.head()
data.shape

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.511,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.721,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.738,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.521,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.617,-122.045,1800,7503


(21613, 21)

Dropeamos los datos de id y date

In [14]:
eliminate = ["id","date"]
data = data.drop(labels=eliminate, axis=1)
data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.511,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.721,-122.319,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.738,-122.233,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.521,-122.393,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.617,-122.045,1800,7503


In [15]:
data.isna().sum()

price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [16]:
data_train,data_test=train_test_split(data,test_size = 0.2, random_state=33)
data_train.shape
data_test.shape 

(17290, 19)

(4323, 19)

In [17]:
data_train_variables = data_train.loc[:,data_train.columns !="price"]
data_train_res = data_train["price"]
data_train_res=data_train_res[data_train_res.index]
data_test_variables = data_test.loc[:,data_test.columns !="price"]
data_test_res = data_test["price"]
data_train_variables.head()
data_train_res.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
4017,3,2.50,3190,49137,2.0,0,0,3,9,3190,0,1988,0,98074,47.603,-122.043,3240,53143
18986,3,2.00,1190,6833,1.0,0,0,3,7,1190,0,1995,0,98092,47.319,-122.180,1540,8000
2149,3,1.75,1390,8700,1.0,0,3,4,7,840,550,1912,0,98055,47.472,-122.202,1390,10875
13330,2,1.00,890,7200,1.0,0,0,3,7,740,150,1949,0,98115,47.700,-122.278,1290,7200
19717,3,2.25,1240,949,3.0,0,0,3,8,1240,0,2008,0,98103,47.649,-122.334,1310,1140


4017     760750.0
18986    200000.0
2149     280950.0
13330    305000.0
19717    543000.0
Name: price, dtype: float64

Hacemos linear regresion

In [18]:
model = sm.OLS(data_train_res, sm.add_constant(data_train_variables))
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.698
Model:                            OLS   Adj. R-squared:                  0.698
Method:                 Least Squares   F-statistic:                     2354.
Date:                Wed, 04 Oct 2023   Prob (F-statistic):               0.00
Time:                        16:53:01   Log-Likelihood:            -2.3535e+05
No. Observations:               17290   AIC:                         4.707e+05
Df Residuals:                   17272   BIC:                         4.709e+05
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          7.246e+06   3.21e+06      2.260      0.024     9.6e+05    1.35e+07
bedrooms      -3.224e+04   2061.133    -15.641      0.000   -3.63e+04   -2.82e+04
bathrooms      4.209e+04   3544.464     11.874      0.000    3.51e+04     4.9e+04
sqft_living     106.2527      2.504     42.430      0.000     101.344     111.161
sqft_lot          0.0973      0.052      1.885      0.059      -0.004       0.198
floors         7256.9032   3937.586      1.843      0.065    -461.164     1.5e+04
waterfront     4.948e+05    1.9e+04     26.092      0.000    4.58e+05    5.32e+05
view           5.485e+04   2363.580     23.206      0.000    5.02e+04    5.95e+04
condition       2.57e+04   2579.149      9.966      0.000    2.06e+04    3.08e+04
grade          9.638e+04   2351.511     40.988      0.000    9.18e+04    1.01e+05
sqft_above       68.6495      2.462     27.887      0.000      63.824      73.475
sqft_basement    37.6012      2.921     12.875      0.000      31.877      43.326
yr_built      -2640.2033     79.278    -33.303      0.000   -2795.597   -2484.810
yr_renovated     21.4730      3.976      5.401      0.000      13.680      29.266
zipcode        -576.0145     36.190    -15.916      0.000    -646.951    -505.078
lat            5.997e+05   1.18e+04     50.778      0.000    5.77e+05    6.23e+05
long          -2.065e+05   1.45e+04    -14.287      0.000   -2.35e+05   -1.78e+05
sqft_living15    20.7640      3.792      5.476      0.000      13.331      28.197
sqft_lot15       -0.3569      0.080     -4.474      0.000      -0.513      -0.201
==============================================================================
Omnibus:                    14726.865   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1648739.270
Skew:                           3.536   Prob(JB):                         0.00
Kurtosis:                      50.313   Cond. No.                     6.53e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.11e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""